In [4]:
import os
%pwd

'c:\\Users\\MEDHA TRUST\\Documents\\Govardhan\\ML\\github\\CustomerChurn\\CustomerChurn\\research'

In [5]:
## command should be in CustomerChurn directory
os.chdir('../')
%pwd

'c:\\Users\\MEDHA TRUST\\Documents\\Govardhan\\ML\\github\\CustomerChurn\\CustomerChurn'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    data_dir: Path
    schema: dict


In [15]:
from ChurnPrediction.constants import *
from ChurnPrediction.utils.common import read_yaml, create_directory

In [16]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directory([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directory([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            data_dir = config.data_dir,
            STATUS_FILE = config.STATUS_FILE,
            schema = schema
        )

        return data_validation_config
    

In [21]:
import os
from ChurnPrediction import logger
import pandas as pd

In [36]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            logger.info("DataValidation started")
            validation_status = None
            data = pd.read_csv(self.config.data_dir)
            all_cols = list(data.drop(['Churn'], axis = 1).columns)
            all_schema_cols = self.config.schema

            for col in all_cols:
                if (col not in all_schema_cols.keys()) or (all_schema_cols[col] != data[col].dtype):
                    #logger.info(f"{col} :  {all_schema_cols[col]} !=  {data[col].dtype}")
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation status: {validation_status}')

                else:
                    #logger.info(f"{col} :  {all_schema_cols[col]} ==  {data[col].dtype}")
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f'Validation status: {validation_status}')
                    
            return validation_status
            
        except Exception as  e:
            raise e

In [37]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()

except Exception as e:
    raise e

[2023-10-15 07:13:24,738: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-15 07:13:24,738: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-15 07:13:24,738: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-15 07:13:24,738: INFO: common: directory: artifacts created successfully]
[2023-10-15 07:13:24,751: INFO: common: directory: artifacts/data_validation created successfully]
[2023-10-15 07:13:24,752: INFO: 4094831145: DataValidation started]
